In [1]:
from pathlib import Path


root_path = Path("/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/")

fpath_output = Path("/home/jovyan/InputData/")
fpath_output.mkdir(exist_ok=True, parents=True)




In [2]:
def get_subdirs(path: Path):
    list_paths = []
    for path in Path(path).iterdir():
        if path.is_dir():
            list_paths.append(path)
    
    return list_paths


dirs_acdc = get_subdirs(root_path)
print(dirs_acdc)


[PosixPath('/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/DCM'), PosixPath('/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/HCM'), PosixPath('/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/MINF'), PosixPath('/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/NOR')]


In [3]:

import nibabel as nib

import numpy as np

def crop_image(imgarr, cropped_size):
    
    imgdims = np.array(imgarr.shape)//2
    xstart = imgdims[0] - cropped_size//2
    xend = imgdims[0] + cropped_size//2
    
    ystart = imgdims[1] - cropped_size//2
    yend = imgdims[1] + cropped_size//2
    

    return imgarr[xstart:xend, ystart:yend,...]

def get_cropped_image(path: Path):
    
    img = nib.load(str(path / "image.nii.gz"))

    cropsize=128
    cropped_arr = crop_image(img.get_fdata(), cropsize)

    return np.squeeze(cropped_arr)

min_num_phases = 24
time_usf = 2

    

In [4]:

i=0
for curr_dir in dirs_acdc:
    print(curr_dir)
    list_patients = get_subdirs(curr_dir)
    
    for pat in list_patients:
        img = get_cropped_image(pat)
        
        if img.shape[3] < min_num_phases:
            continue
        
        img = img[...,1:min_num_phases:time_usf]
        
        for islice in range(img.shape[2]):
            dat = np.squeeze(img[:,:,islice,:])
            
            dat = np.swapaxes(dat, 0, 2)
            nii = nib.Nifti1Image(dat, np.eye(4))
            
            print("Storing img number {}".format(i))
            nib.save(nii, str(fpath_output / "img_{}.nii".format(i)))
            i += 1
        

/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/DCM
Storing img number 0
Storing img number 1
Storing img number 2
Storing img number 3
Storing img number 4
Storing img number 5
Storing img number 6
Storing img number 7
Storing img number 8
Storing img number 9
Storing img number 10
Storing img number 11
Storing img number 12
Storing img number 13
Storing img number 14
Storing img number 15
Storing img number 16
Storing img number 17
Storing img number 18
Storing img number 19
Storing img number 20
Storing img number 21
Storing img number 22
Storing img number 23
Storing img number 24
Storing img number 25
Storing img number 26
Storing img number 27
Storing img number 28
Storing img number 29
Storing img number 30
Storing img number 31
Storing img number 32
Storing img number 33
Storing img number 34
Storing img number 35
Storing img number 36
Storing img number 37
Storing img number 38
Storing img number 39
Storing img number 40
Storing img number 41
Storing img number 42
Storing i

In [7]:
import sirf.Gadgetron as pMR
import numpy as np


fname_template = fpath_output / "template/cine_128_30ph_bc.h5"
template_data = pMR.AcquisitionData(str(fname_template))

template_data = pMR.preprocess_acquisition_data(template_data)
template_phase = template_data.get_ISMRMRD_info("phase")

index_all_acquis = np.arange(template_data.number())

index_interesting_acquis = template_phase < (min_num_phases// time_usf)
index_interesting_acquis = index_all_acquis[index_interesting_acquis]
template_data = template_data.get_subset(index_interesting_acquis)



Started reading acquisitions from /home/jovyan/InputData/template/cine_128_30ph_bc.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..100%..
Finished reading acquisitions from /home/jovyan/InputData/template/cine_128_30ph_bc.h5


In [8]:
import mathplus_p1_auxiliary_functions as aux 

def undersample_cartesian_data(ad: pMR.AcquisitionData):

    ky_index = ad.get_ISMRMRD_info('kspace_encode_step_1')
    cph_index = ad.get_ISMRMRD_info('phase')
    ky_num = int(np.max(ky_index)+1)
    cph_num = int(np.max(cph_index)+1)
    print(f'Nky {ky_num} - Ncph {cph_num}')

    R = 4
    F = int(ky_num/10)
    msk = aux.cartesian_mask([cph_num, ky_num, 1], R, sample_n=F)


    acq_us = ad.new_acquisition_data(empty=True)

    # Create raw data
    for cnd in range(cph_num):
        for ynd in range(ky_num):
            if msk[cnd, ynd, 0] == 1:
                cidx = np.where((ky_index == ynd) & (cph_index == cnd))[0]
                if len(cidx) > 0:
                    cacq = ad.acquisition(cidx)
                    acq_us.append_acquisition(cacq)
                else:
                    print(f'ky {ynd} - cph {cnd} not found')

    acq_us.sort()     

    return acq_us

print(template_data.number())
template_data = undersample_cartesian_data(template_data)
print(template_data.number())

1537
Nky 128 - Ncph 12
384


In [11]:
import sirf.Reg as pReg


csm = pMR.CoilSensitivityData()
csm.calculate(template_data)
csm.fill(1 + 0. * 1j)

template_img = pMR.ImageData()
template_img.from_acquisition_data(template_data)

E = pMR.AcquisitionModel(acqs=template_data, imgs=template_img)
E.set_coil_sensitivity_maps(csm)

path_input = fpath_output

list_stacks = sorted(path_input.glob("img*"))

template_fwd = pMR.ImageData()
template_fwd.from_acquisition_data(template_data)

iy = 0
for f in list_stacks:
    print("GO {} , {}/{}!".format(f, iy, len(list_stacks)))
    
    fileidx = str(f.stem).split('_')[1]
        
    nii = pReg.ImageData(str(f))
    template_fwd.fill(nii.as_array())
    
    nii = nib.Nifti1Image(np.abs(template_fwd.as_array()), np.eye(4))
    nib.save(nii, str("/home/jovyan/tmp_{}.nii".format(iy)))

    rd = E.forward(template_fwd)
    
    fout = fpath_output / "y_{}.h5".format(fileidx)
    rd.write(str(fout))
    iy += 1

GO /home/jovyan/InputData/img_0.nii , 0/488!
GO /home/jovyan/InputData/img_1.nii , 1/488!
GO /home/jovyan/InputData/img_10.nii , 2/488!
GO /home/jovyan/InputData/img_100.nii , 3/488!
GO /home/jovyan/InputData/img_101.nii , 4/488!
GO /home/jovyan/InputData/img_102.nii , 5/488!
GO /home/jovyan/InputData/img_103.nii , 6/488!
GO /home/jovyan/InputData/img_104.nii , 7/488!
GO /home/jovyan/InputData/img_105.nii , 8/488!
GO /home/jovyan/InputData/img_106.nii , 9/488!
GO /home/jovyan/InputData/img_107.nii , 10/488!
GO /home/jovyan/InputData/img_108.nii , 11/488!
GO /home/jovyan/InputData/img_109.nii , 12/488!
GO /home/jovyan/InputData/img_11.nii , 13/488!
GO /home/jovyan/InputData/img_110.nii , 14/488!
GO /home/jovyan/InputData/img_111.nii , 15/488!
GO /home/jovyan/InputData/img_112.nii , 16/488!
GO /home/jovyan/InputData/img_113.nii , 17/488!
uld cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause

In [12]:
fname_test = fpath_output / "y_55.h5"
ad_test = pMR.AcquisitionData(str(fname_test))

img_test = E.inverse(ad_test)
nii = nib.Nifti1Image(np.abs(img_test.as_array()), np.eye(4))
nib.save(nii, str(fpath_output / "pseudoinv.nii"))

pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per